# Foundations of AI/ML by IIIT-Hyderabad & Talent Sprint
# Lab10 Experiment 02

## Visualizing CNNs ##

In this experiment, we will first follow the approach used by [Matthew D Zeiler, Rob Fergus](https://arxiv.org/abs/1311.2901) to visualize CNNs. We will visualize a VGG16 model trained on the Imagenet dataset.

We also follow the approach used by [Mahendran and Vedaldi](https://arxiv.org/abs/1412.0035) to understand deep image representations. [Section 6 onwards]

## 1. Load the model and see its architecture

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import models
from torchvision import transforms, utils

import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

model = models.vgg16(pretrained=True)
print(model.features)

### We will load all the module details in a list 

In [ ]:
modules = list(model.features.modules())
modules = modules[1:]
print(modules,"\n\n")
print("third module = ", modules[2])

### 2. Load and preprocess an image to pass as input to the network

In [ ]:
def normalize(image):
    normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
    )
    preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
    ])
    image = Variable(preprocess(image).unsqueeze(0))
    return image

img_raw = Image.open("./dog.jpg")
plt.imshow(img_raw)
plt.title("Image loaded successfully")

img = normalize(img_raw)

### 3. Visualize the kernel weights at each layer

In [ ]:
def visualize_weights(image, layer):
    weight_used = []
    
    ## Gather all Convolution layers and append their corresponding filters in a list
    for w in model.features.children():
        if isinstance(w, torch.nn.modules.conv.Conv2d):
            weight_used.append(w.weight.data)

    print("(#filters, i/p depth, size of filter) === ",weight_used[layer].shape)
    print("No. of filters: ", weight_used[layer].shape[0])
    filters = []
    for i in range(weight_used[layer].shape[0]):
        filters.append(weight_used[layer][i,:,:,:].sum(dim=0))    ##summing across input depth(3 in the first layer)
        filters[i].div(weight_used[layer].shape[1])
        
    fig = plt.figure()
    plt.rcParams["figure.figsize"] = (10, 10)
    for i in range(int(np.sqrt(weight_used[layer].shape[0])) * int(np.sqrt(weight_used[layer].shape[0]))):
        a = fig.add_subplot(np.sqrt(weight_used[layer].shape[0]),np.sqrt(weight_used[layer].shape[0]),i+1)
        imgplot = plt.imshow(filters[i])
        plt.axis('off')

visualize_weights(img, 1)

The filters are usually small $(3X3)$ and hence the visualization of filter weights usually doesn't give us a clear understanding of the what the filters learn.
 
Therefore, we will visualize how the the input image looks as it is passed through the various layers in the network.

### 4. Visualizing the image as it passes through the network

In [ ]:
def to_grayscale(image):
    image = torch.sum(image, dim=0)
    image = torch.div(image, image.shape[0])
    return image

def layer_outputs(image):
    outputs = []
    names = []
    
    ## feed forward the image through the network and store the outputs
    for layer in modules:
        image = layer(image) 
        outputs.append(image)
        names.append(str(layer))
    
    ## for visualization purposes, convert the output into a 2D image by averaging across the filters.
    output_im = []
    for i in outputs:
        i = i.squeeze(0)
        temp = to_grayscale(i)  ## convert say 64x112x112 to 112x112
        output_im.append(temp.data.numpy())
        
    fig = plt.figure()
    plt.rcParams["figure.figsize"] = (30, 40)


    for i in range(len(output_im)):
        a = fig.add_subplot(8,4,i+1)
        imgplot = plt.imshow(output_im[i])
        plt.axis('off')
        a.set_title(str(i+1)+". "+names[i].partition('(')[0], fontsize=30)

#     ##save the resulting visualization
#     plt.savefig('layer_outputs.jpg', bbox_inches='tight')

##
layer_outputs(img)

Through the above visualization, it is clearly visible how the CNN responds to an image at each layer and in the final layer, the pixels of the image which produce the highest activation values are visible as well. However, we still haven't looked at how the filters at each layer respond differently to features present in the input.

### 5. Visualizing output of each filter at a given layer 

In [ ]:
def filter_outputs(image, layer_to_visualize, num_filters=64):
    if layer_to_visualize < 0:
        layer_to_visualize += 31
    output = None
    name = None
    
    ## get outputs corresponding to the mentioned layer
    for count, layer in enumerate(modules):
        image = layer(image)
        if count == layer_to_visualize: 
            output = image
            name = str(layer)
    
    filters = []
    output = output.data.squeeze()

    ## if num_filters==-1, visualize all the filters
    num_filters = min(num_filters, output.shape[0])
    if num_filters==-1:
        num_filters = output.shape[0]

    for i in range(num_filters):
        filters.append(output[i,:,:])
        
    fig = plt.figure()
    plt.rcParams["figure.figsize"] = (10, 10)

    for i in range(int(np.sqrt(len(filters))) * int(np.sqrt(len(filters)))):
        fig.add_subplot(np.sqrt(len(filters)), np.sqrt(len(filters)),i+1)
        imgplot = plt.imshow(filters[i])
        plt.axis('off')

## if num_filters==-1, visualize all the filters
filter_outputs(img,0,16)    #visualize the outputs of first 16 filters of the 1st layer

The above visualization shows that each filter responds differently to an input which implies that each filter learns and extracts different features from an input.

### 6. Understanding Deep Image Representations by Inverting Them [Mahendran, Vedaldi]

Like Zeiler and Fergus, their method starts from a specific input image. They record the network’s representation of that specific image and then reconstruct an image that produces a similar code. Thus, their method provides insight into what the activation of a whole layer represent, not what an individual neuron represents.

They show what each neuron “wants to see”, and thus what each neuron has learned to look for.

To visualize the function of a specific unit in a neural network, we $synthesize$ inputs that cause that unit to have high activation. To synthesize such a “preferred input example”, we start with a random image, meaning we randomly choose a color for each pixel. The image will initially look like colored TV static.

In [ ]:
random_noise_img = Variable(1e-1 * torch.randn(1, 3, 224, 224), requires_grad=True)

Now we take an image $X$ whose representation $X_0$ at some layer $"target\_layer"$ we want to learn. Our aim is to reconstruct the noise image to get this representation $X_0$. The principle behind this is that the noise image will be so reconstructed such that it will represent what the particular layer for which it is trained against wants to see.

In [ ]:
def get_output_at_nth_layer(inp, layer):
    for i in range(layer):
        inp = modules[i](inp)
    return inp[0]

## dont forget that the system is 0 indexed
target_layer = 18    ## which is this layer Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
inp_img = normalize(Image.open("./dog.jpg"))
inp_img_representation = get_output_at_nth_layer(inp_img, target_layer)

Some functions of code (euclidean_loss, alpha_norm, total_variation_norm, recreate_image) are present in utils.py. If you truly want to understand how this is implemented, it is recommended that you readthe second and third page of this [paper](https://arxiv.org/abs/1412.0035), specifically, the regularization part, before asking questions on that. The aim of this code is to mainly understand the deep representations.

In [ ]:
from torch.optim import SGD
import utils

#define optimizers for learning the representation of the noise input image
optimizer = SGD([random_noise_img], lr=1e4, momentum=0.9)
alpha_reg_alpha = 1
alpha_reg_lambda = 1e-7
tv_reg_beta = 2
tv_reg_lambda = 1e-8
## Put model into evaluation state
model.eval

In [ ]:
imgs=[]
for i in range(161):
        optimizer.zero_grad()
        
        ## get output at the target layer (not the final layer)
        output = get_output_at_nth_layer(random_noise_img,target_layer)
        
        # Calculate euclidian loss between output image and the target image
        euc_loss = 1e-1 * utils.euclidian_loss(inp_img_representation.detach(), output)
        
        # regularization
        reg_alpha = alpha_reg_lambda * utils.alpha_norm(random_noise_img, alpha_reg_alpha)
        reg_total_variation = tv_reg_lambda * utils.total_variation_norm(random_noise_img,tv_reg_beta)
        
        loss = euc_loss + reg_alpha + reg_total_variation
        loss.backward()
        optimizer.step()
        
        # Generate image every 5 iterations
        if i % 10 == 0:
            print('Iteration:', str(i), 'Loss:', loss.data.numpy()[0])
            x = utils.recreate_image(random_noise_img)
            imgs.append(x)

### Visualize the learned representations for the layer "target_layer"

In [ ]:
fig = plt.figure()
plt.rcParams["figure.figsize"] = (10, 10)
for i in range(int(np.sqrt(len(imgs))) * int(np.sqrt(len(imgs)))):
    a = fig.add_subplot(np.sqrt(len(imgs)), np.sqrt(len(imgs)),i+1)
    imgplot = plt.imshow(imgs[i])
    a.set_title("iter = "+ str((i+1)*10))
    plt.axis('off')

### Exercise: Above we visualized the inverted representations for the 18th layer, find similar representations for different layers in the network and thus visualize what the network learns at each of those layer. Try for different target images as well